In this notebook we'll be using transformerlens and runpods to generate outputs 

I'll do this under head ablation

runpods questions 

do I have a credit card number to use for runpods?

how do I make sure runpods start when I want them and stop when im finished?

In [1]:
from transformer_lens import HookedTransformer
import transformer_lens
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import transformer_lens.utils as utils
import hashlib
import yaml 
import hashlib
import pickle
import numpy as np
import matplotlib.pyplot as plt 

/root/mech_interp_research/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pickle

# Save object to a file
def save_pickle(obj, filepath):
    with open(filepath, 'wb') as f:
        pickle.dump(obj, f)

# Load object from a file
def load_pickle(filepath):
    with open(filepath, 'rb') as f:
        return pickle.load(f)
    

In [3]:
device = 'cuda:0'

In [4]:
reference_model_path = 'meta-llama/Llama-3.1-8B'
baseline_model_path = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

In [5]:
baseline_model_hf = AutoModelForCausalLM.from_pretrained(baseline_model_path, torch_dtype=torch.bfloat16)
baseline_model_tokenizer = AutoTokenizer.from_pretrained(baseline_model_path)

model = HookedTransformer.from_pretrained_no_processing(
    reference_model_path,
    hf_model=baseline_model_hf,
    tokenizer=baseline_model_tokenizer,
    device=device,
    move_to_device=True,
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  5.35it/s]


Loaded pretrained model meta-llama/Llama-3.1-8B into HookedTransformer


In [6]:
model

HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (blocks): ModuleList(
    (0-31): 32 x TransformerBlock(
      (ln1): RMSNorm(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): RMSNorm(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): GroupedQueryAttention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
        (hook_rot_k): HookPoint()
        (hook_rot_q): HookPoint()
      )
      (mlp): GatedMLP(
        (hook_pre): HookPoint()
        (hook_pre_linear): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_in): HookPoint()
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_attn_out)

In [7]:
def produce_head_ablation(head_idx):
    def head_ablation_hook(
        value,
        hook
    ):
        # print(f"Shape of the value tensor: {value.shape}")
        value[:, :, head_idx, :] = 0.
        return value

    return head_ablation_hook


def generate_using_ablated_head(model, init_strings, ablate_heads=None, max_new_tokens=300, format_init_strings=True, save_as_you_go=None):
    """ 
    
    """
    if format_init_strings:
        init_strings=[f"<｜User｜>{string}<｜Assistant｜><think>\n" for string in init_strings]
    
    # apply ablations
    model.reset_hooks()
    if ablate_heads is not None:
        for layer_idx, head_idx in ablate_heads:
            cur_head_ablation = produce_head_ablation(head_idx)
            model.add_hook(utils.get_act_name("z", layer_idx), cur_head_ablation)

        print("ablate heads:")
        for head in ablate_heads:
            print(ablate_heads)
    
    if save_as_you_go is not None:
        assert isinstance(save_as_you_go, str)
        if '.pkl' != save_as_you_go[-4:]:
            save_as_you_go = save_as_you_go+".pkl"

    # now generate 
    # these are the default settings according 
    # https://huggingface.co/deepseek-ai/DeepSeek-R1-Distill-Llama-8B
    temp=0.6 
    top_p=0.95 

    all_generated_outs = []
    for init_string in init_strings:
        test_tokens = model.to_tokens(init_string)
        generated_outputs = model.generate(test_tokens, max_new_tokens=max_new_tokens, temperature=temp, top_p=top_p)

        all_generated_outs.append(generated_outputs)

        if save_as_you_go is not None:
            save_pickle(all_generated_outs, save_as_you_go)
    
    return all_generated_outs


load the collected think dict and ablate groups of important think heads 

In [8]:
with open('collected_think_dict.yaml', 'r') as file:
    think_dict = yaml.safe_load(file)

# ranking head importance 

In [9]:
head_str, mean_think_stop_think_ratios = zip(
    *sorted(
        [(k, think_dict[k]['mean_think_stop_think_ratio']) for k in think_dict],
        key = lambda x: x[-1],
        reverse=True
    )
)

In [10]:
for i in range(10):
    print(head_str[i], mean_think_stop_think_ratios[i])

0_22 8.45154857635498
3_1 6.1095075607299805
0_21 4.9486775398254395
4_21 4.679234027862549
8_18 4.370467185974121
1_23 4.120393753051758
1_12 3.7947165966033936
1_22 3.7929306030273438
16_20 3.6640069484710693
7_21 3.477773666381836


We'll take the top 4 important heads and ablate them one by one 

In [11]:
top_4_targets = [tuple([int(x) for x in h.split('_')]) for h in head_str[:4]]

In [12]:
top_4_targets

[(0, 22), (3, 1), (0, 21), (4, 21)]

In [13]:

def read_prompts(file_path="reasoning-prompts.md"):
    """
    Read prompts from a text file and return them as a list of strings.
    
    Args:
        file_path (str): Path to the file containing reasoning prompts
        
    Returns:
        list: A list of strings, each containing a reasoning prompt
    """
    prompts = []
    
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                # Skip empty lines, headers, and category labels
                line = line.strip()
                if (line and 
                    not line.startswith('#') and 
                    not line.startswith('##') and
                    not line == ""):
                    
                    # Extract the prompt text by removing the number and period
                    parts = line.split('. ', 1)
                    if len(parts) > 1 and parts[0].isdigit():
                        prompt = parts[1]
                    else:
                        prompt = line
                        
                    prompts.append(prompt)
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
    except Exception as e:
        print(f"Error reading prompts: {e}")
    
    return prompts

In [14]:
sampled_prompts = read_prompts()[:50]

In [15]:
collected_outputs = {}

for i in range(4):
    save_string = f"top_{i+1}_heads_ablated.pkl"

    generated_outputs = generate_using_ablated_head(
        model, 
        init_strings=sampled_prompts, 
        ablate_heads=top_4_targets[:i+1], 
        max_new_tokens=300, 
        format_init_strings=True, 
        save_as_you_go=save_string
    )

    collected_outputs[save_string]=generated_outputs

ablate heads:
[(0, 22)]


  0%|          | 0/300 [00:00<?, ?it/s]

 87%|████████▋ | 262/300 [00:19<00:02, 13.58it/s]


ablate heads:
[(0, 22), (3, 1)]
[(0, 22), (3, 1)]


100%|██████████| 300/300 [00:22<00:00, 13.60it/s]


ablate heads:
[(0, 22), (3, 1), (0, 21)]
[(0, 22), (3, 1), (0, 21)]
[(0, 22), (3, 1), (0, 21)]


100%|██████████| 300/300 [00:22<00:00, 13.62it/s]


ablate heads:
[(0, 22), (3, 1), (0, 21), (4, 21)]
[(0, 22), (3, 1), (0, 21), (4, 21)]
[(0, 22), (3, 1), (0, 21), (4, 21)]
[(0, 22), (3, 1), (0, 21), (4, 21)]


 90%|█████████ | 271/300 [00:20<00:02, 13.54it/s]


In [16]:
generated_outputs = generate_using_ablated_head(
    model, 
    init_strings=sampled_prompts, 
    ablate_heads=None, 
    max_new_tokens=300, 
    format_init_strings=True, 
    save_as_you_go="no_heads_ablated.pkl"
)

 10%|▉         | 29/300 [00:02<00:20, 13.49it/s]


KeyboardInterrupt: 

In [ ]:
jhg

In [ ]:
tokens = model.to_tokens("Hey whats up?")

In [ ]:
tokens

tensor([[128000,  19182,  41209,    709,     30]], device='cuda:0')

In [ ]:

model.reset_hooks()
original_logits = model(tokens, return_type="logits")

test_tokens = model.to_tokens(["Hey whats up?"])

# try generating 
outs = model.generate(test_tokens, max_new_tokens=400, temperature=0.6, top_p=0.95)

100%|██████████| 400/400 [00:29<00:00, 13.60it/s]


In [ ]:
model.to_string(outs)

["<｜begin▁of▁sentence｜>Hey whats up? So I just saw this video where someone was trying to make a cake from scratch, but it didn't turn out so well. It was kinda funny but also kind of sad because the person put so much effort into it. I wonder why it didn't work out. Maybe they didn't measure the ingredients properly? Or did they forget to add baking powder? I mean, I've had my fair share of baking failures before, so I can relate. Let me think about how to approach this.\n\nFirst off, when I try baking, I usually just throw in whatever I have, but that's probably why my cakes sometimes end up too dense or too dry. Maybe I should double-check the recipes next time. But wait, the person in the video was following a recipe, right? So maybe the issue was with the measurements. Did they use the right amounts of flour, sugar, butter, etc.? It's easy to get confused with cup measurements versus grams or milliliters, especially if you're not used to it.\n\nAlso, could it be the oven temperatu

In [ ]:
utils.get_act_name("z", 0)

'blocks.0.attn.hook_z'

In [ ]:
model.reset_hooks()

In [ ]:

model.add_hook(utils.get_act_name("z", layer_to_ablate), head_ablation_hook)


modified_logits = model(tokens, return_type="logits")

Shape of the value tensor: torch.Size([1, 5, 32, 128])


In [ ]:
modified_logits

tensor([[[ 1.2196,  2.1443,  4.9758,  ..., -1.1626, -1.1626, -1.1626],
         [12.2630,  1.2126,  0.5368,  ..., -3.3336, -3.3335, -3.3338],
         [ 5.2130,  2.5234, -0.0555,  ..., -4.4290, -4.4293, -4.4291],
         [11.5725,  3.8210,  1.2050,  ..., -5.2874, -5.2878, -5.2877],
         [ 5.1244, -3.3234,  1.2445,  ..., -4.2034, -4.2034, -4.2036]]],
       device='cuda:0', grad_fn=<ViewBackward0>)

In [ ]:
original_logits

tensor([[[ 1.1601,  2.0860,  4.8631,  ..., -1.1549, -1.1549, -1.1549],
         [12.2188,  1.1821,  0.5339,  ..., -3.3060, -3.3059, -3.3062],
         [ 5.2511,  2.6107, -0.0394,  ..., -4.4606, -4.4609, -4.4606],
         [11.5757,  3.8677,  1.2013,  ..., -5.2869, -5.2873, -5.2872],
         [ 5.1116, -3.3044,  1.2272,  ..., -4.2133, -4.2133, -4.2136]]],
       device='cuda:0', grad_fn=<ViewBackward0>)

In [ ]:
model.reset_hooks()
original_logits2 = model(tokens, return_type="logits")


In [ ]:
original_logits2

tensor([[[ 1.1601,  2.0860,  4.8631,  ..., -1.1549, -1.1549, -1.1549],
         [12.2188,  1.1821,  0.5339,  ..., -3.3060, -3.3059, -3.3062],
         [ 5.2511,  2.6107, -0.0394,  ..., -4.4606, -4.4609, -4.4606],
         [11.5757,  3.8677,  1.2013,  ..., -5.2869, -5.2873, -5.2872],
         [ 5.1116, -3.3044,  1.2272,  ..., -4.2133, -4.2133, -4.2136]]],
       device='cuda:0', grad_fn=<ViewBackward0>)